# Assignment 1: 

## Question 1: Value Iteration Implementation

In [3]:
import numpy as np

# ----- Environment and Parameters Setup -----
grid_size = 5                   # Define grid size: 5x5 grid
goal = (4, 4)                   # Specify goal state (bottom-right corner)
gamma = 0.9                     # Discount factor for future rewards
reward_goal = 10                # Reward for reaching the goal state
reward_step = -1                # Penalty for making any non-goal step

# Create a value function table (matrix) initialized to zeros
V = np.zeros((grid_size, grid_size))

# Define the possible actions and their effects on grid coordinates.
# 'U' = Up, 'D' = Down, 'L' = Left, 'R' = Right.
actions = {
    'U': (-1, 0),  # Move up: decrease row index by 1
    'D': (1, 0),   # Move down: increase row index by 1
    'L': (0, -1),  # Move left: decrease column index by 1
    'R': (0, 1)    # Move right: increase column index by 1
}

def is_valid_state(x, y):
    """Check if (x, y) is a valid state within the grid boundaries."""
    return 0 <= x < grid_size and 0 <= y < grid_size

# Set a small threshold epsilon for determining convergence
epsilon = 1e-3
iteration = 0  # To keep track of the number of iterations

# ----- Value Iteration Process -----
while True:
    delta = 0  # Initialize the maximum change in value for this iteration
    new_V = np.copy(V)  # Copy the current value function to update it
    
    # Iterate over each state in the grid world
    for i in range(grid_size):
        for j in range(grid_size):
            # Skip updating the value for the goal state since it is terminal.
            if (i, j) == goal:
                continue
            
            values = []  # List to store computed values for each possible action
            
            # Evaluate all possible actions from state (i, j)
            for a in actions:
                dx, dy = actions[a]             # Get the effect of the action 'a'
                new_i, new_j = i + dx, j + dy     # Calculate new state coordinates
                
                # If the new state is off the grid, remain in the same state.
                if not is_valid_state(new_i, new_j):
                    new_i, new_j = i, j
                    
                # Assign immediate reward:
                # +10 if the new state is the goal, otherwise -1.
                r = reward_goal if (new_i, new_j) == goal else reward_step
                
                # Compute the value: immediate reward plus discounted value of the resulting state
                value = r + gamma * V[new_i, new_j]
                values.append(value)
            
            best_value = max(values)         # Determine the best achievable value from state (i, j)
            new_V[i, j] = best_value           # Update the value for state (i, j)
            
            # Update delta with the maximum change observed for convergence checking.
            delta = max(delta, abs(new_V[i, j] - V[i, j]))
    
    V = new_V  # Update the value function matrix for the next iteration
    iteration += 1
    
    # When the maximum change is less than epsilon, we've converged.
    if delta < epsilon:
        break

# ----- Policy Extraction -----
# Create a policy grid with the same dimensions to store the best action for each state.
policy = np.full((grid_size, grid_size), '', dtype=object)
for i in range(grid_size):
    for j in range(grid_size):
        # For the goal state, mark it explicitly.
        if (i, j) == goal:
            policy[i, j] = 'G'
        else:
            best_action = None
            best_value = float('-inf')
            # Evaluate every action to determine the best one.
            for a in actions:
                dx, dy = actions[a]
                new_i, new_j = i + dx, j + dy
                if not is_valid_state(new_i, new_j):
                    new_i, new_j = i, j
                r = reward_goal if (new_i, new_j) == goal else reward_step
                value = r + gamma * V[new_i, new_j]
                if value > best_value:
                    best_value = value
                    best_action = a
            policy[i, j] = best_action

# ----- Print the Optimal Policy -----
print("Optimal Policy Grid (Value Iteration):")
for row in policy:
    print(row)

Optimal Policy Grid (Value Iteration):
['D' 'D' 'D' 'D' 'D']
['D' 'D' 'D' 'D' 'D']
['D' 'D' 'D' 'D' 'D']
['D' 'D' 'D' 'D' 'D']
['R' 'R' 'R' 'R' 'G']


## Question 2: Q-learning Implementation


In [4]:
import numpy as np
import random

# ----- Environment and Parameters Setup -----
grid_size = 5                       # Define grid size: 5x5 grid world
goal = (4, 4)                       # Specify goal state
reward_goal = 10                    # Reward for reaching the goal state
reward_step = -1                    # Penalty for taking any non-goal step

# Define the set of possible actions in an ordered list.
actions_list = ['U', 'D', 'L', 'R']  # List of actions: up, down, left, right
# Mapping from actions to the effect on grid coordinates.
action_effects = {
    'U': (-1, 0),   # Up: move one row up
    'D': (1, 0),    # Down: move one row down
    'L': (0, -1),   # Left: move one column left
    'R': (0, 1)     # Right: move one column right
}

def is_valid_state(x, y):
    """Return True if the state (x, y) lies within the grid boundaries."""
    return 0 <= x < grid_size and 0 <= y < grid_size

# ----- Q-learning Parameters -----
alpha = 0.5             # Learning rate determines how new experiences affect Q-values
gamma = 0.9             # Discount factor for future rewards
epsilon = 0.1           # Exploration rate for ε-greedy policy
episodes = 1000         # Number of episodes for training the agent
max_steps = 100         # Maximum steps per episode to avoid infinite loops

# Initialize the Q-table with zeros.
# The table size is (grid_size x grid_size x number of actions).
Q = np.zeros((grid_size, grid_size, len(actions_list)))

# ----- Q-learning Algorithm -----
for episode in range(episodes):
    state = (0, 0)  # Start each episode from the defined starting state (0, 0)
    
    for step in range(max_steps):
        i, j = state  # Unpack current state's coordinates
        
        # ε-greedy policy: decide whether to explore or exploit.
        if random.uniform(0, 1) < epsilon:
            # Exploration: choose a random action from the available actions.
            action_index = random.randint(0, len(actions_list) - 1)
        else:
            # Exploitation: choose the best action based on the current Q-value estimates.
            action_index = np.argmax(Q[i, j])
        action = actions_list[action_index]  # Get the chosen action symbol
        
        # Determine the new state after taking the chosen action.
        dx, dy = action_effects[action]
        new_i, new_j = i + dx, j + dy
        # If the new state is off the grid, the agent remains in the current state.
        if not is_valid_state(new_i, new_j):
            new_i, new_j = i, j
        
        next_state = (new_i, new_j)
        
        # Assign immediate reward: +10 if the new state is the goal; otherwise -1.
        reward = reward_goal if next_state == goal else reward_step
        
        # ----- Q-learning Update -----
        # Fetch current Q-value for state-action pair Q(s,a)
        current_q = Q[i, j, action_index]
        # Estimate maximum future Q-value for the next state: max_a' Q(s', a')
        max_q_next = np.max(Q[new_i, new_j])
        # Update the Q-value using the Q-learning update rule.
        Q[i, j, action_index] = current_q + alpha * (reward + gamma * max_q_next - current_q)
        
        # Transition to the next state.
        state = next_state
        
        # If the agent reaches the goal, terminate the episode early.
        if state == goal:
            break

# ----- Print the Learned Q-values for Each State -----
print("\nLearned Q-values (Q-learning):")
# Loop over every state in the grid world
for i in range(grid_size):
    for j in range(grid_size):
        print(f"State ({i},{j}):")
        # Display the Q-value for each action in the current state.
        for index, action in enumerate(actions_list):
            print(f"  Action {action}: {Q[i, j, index]:.2f}")
    print()  # Add an empty line for better readability between grid rows


Learned Q-values (Q-learning):
State (0,0):
  Action U: -1.39
  Action D: -0.43
  Action L: -1.39
  Action R: -0.43
State (0,1):
  Action U: -1.05
  Action D: 0.63
  Action L: -1.58
  Action R: -0.32
State (0,2):
  Action U: -2.26
  Action D: 1.81
  Action L: -0.69
  Action R: -2.16
State (0,3):
  Action U: -1.85
  Action D: 3.12
  Action L: -1.91
  Action R: -1.54
State (0,4):
  Action U: 1.93
  Action D: 4.58
  Action L: -1.59
  Action R: -1.43

State (1,0):
  Action U: -1.39
  Action D: 0.63
  Action L: -0.43
  Action R: 0.63
State (1,1):
  Action U: -0.43
  Action D: 1.81
  Action L: -0.43
  Action R: 1.81
State (1,2):
  Action U: 0.63
  Action D: 3.12
  Action L: 0.63
  Action R: 3.12
State (1,3):
  Action U: 1.81
  Action D: 1.81
  Action L: 1.81
  Action R: 4.58
State (1,4):
  Action U: 3.12
  Action D: 6.20
  Action L: 3.12
  Action R: 4.58

State (2,0):
  Action U: -0.44
  Action D: -2.42
  Action L: -2.54
  Action R: 1.81
State (2,1):
  Action U: 0.31
  Action D: 3.12
  Acti